In [3]:
def anti_trace_map(point, d, p, E):
    return d * point - trace_map(point, d, p, E)

def trace_map(point, d, p, E):
    result = point
    point_t = point
    for i in range(1, d):
        point_x, point_y = list(point_t)[0], list(point_t)[1]
        point_t = E(point_x ** p, point_y ** p)
        result = result + point_t
    return result

In [4]:
p = 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559787
q = 52435875175126190479447740508185965837690552500527637822603658699938581184513

A = 0
B = 4

## base prime field
Fp = GF(p)

## E1 over base prime field, map any point on Efp into the q-torsion subgroup
Efp = EllipticCurve(Fp, [A, B])
r_E = Efp.order()
cofactor_E1 = r_E // q
# g_E1 = Efp(0)
# while g_E1 == Efp(0):
#     a = Efp.random_element()
#     g_E1 = cofactor * a
g_E1 = Efp(
    2262513090815062280530798313005799329941626325687549893214867945091568948276660786250917700289878433394123885724147,
    3165530325623507257754644679249908411459467330345960501615736676710739703656949057125324800107717061311272030899084
)
assert(q * g_E1 == Efp(0))
## trace map on E1 is trival, stays on E1
assert(trace_map(g_E1, 12, p, Efp) == 12 * g_E1)

print('\n ##################################### Curve G1: \n cofactor = {}, \n generator = {}, \n order = {} \n'.format(cofactor_E1, g_E1, r_E))


 ##################################### Curve G1: 
 cofactor = 76329603384216526031706109802092473003, 
 generator = (2262513090815062280530798313005799329941626325687549893214867945091568948276660786250917700289878433394123885724147 : 3165530325623507257754644679249908411459467330345960501615736676710739703656949057125324800107717061311272030899084 : 1), 
 order = 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129030796414117214202539 



In [6]:
########## Fp2 = Fp[X] / X^2 - alpha
## alpha = -1
d = 2
alpha = Fp(-1)
X = Fp['X'].gen()
pol2 = X ** d - alpha
assert(pol2.is_irreducible() == True)
Fp2 = GF(p ** d, 'u', modulus = pol2)
u = Fp2.gen()

## Fp12 = Fp2[X] / X^6 - beta
d = 6
beta = u + 1
XX = Fp2['XX'].gen()
pol12 = XX ** d - beta
assert(pol2.is_irreducible() == True)
beta_t = beta 
Efp2_t = EllipticCurve(Fp2, [A, B * beta_t])
## find the proper twisted curve, who has a q-torsion subgroup which is isomorphism with Efpk's one
if Efp2_t.order() % q != 0:
    beta_t = beta ** 5
    Efp2_t = EllipticCurve(Fp2, [A, B * beta_t])

In [7]:
## twist curve E' over Fp12
Fp12_t = Fp2.extension(pol12, 'x')
Efp12_t = Efp2_t.change_ring(Fp12_t)
print('\n Twist curve E defined over Fp12: {}\n'.format(Efp12_t))


 Twist curve E defined over Fp12: Elliptic Curve defined by y^2 = x^3 + (4*u+4) over Univariate Quotient Polynomial Ring in x over Finite Field in u of size 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559787^2 with modulus x^6 + 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559786*u + 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559786



In [6]:
def rth_root(r, r_co, f):
    # assert(f ** r_co == F(1))
    g = gcd(r, r_co)
    assert(g[0] == 1)
    
    ## r' r = 1 mod h
    ## when sign > 0, v * r - u * M = 1
    ## when sign < 0, u * M - v * r = 1
    if g[2]:
        r_inv = r_co - g[1]
    else:
        r_inv = g[1] 
    assert((r * r_inv) % r_co == 1)
    
    root = pow(f, r_inv, _p)
    assert(pow(root, r, _p) == f)
    
    return root


In [7]:
def rand():
    return randrange(0, _p)

In [8]:
gcd(r, h)

(1,
 15261497220449274157758062739481144524138664264454727802556312953310749764017027395786331591518505975038368957169295615381800600781708243988404195544338605069595970588389975977292772247785491308732534994177528464794307189757107405246489267970952703236847636286243174111510206082507802319795025608328095432685337154348295307337366761316594420276184448843597852643541295902428646973684032044504878457062501361270727944571664422418849766872654843663195009841301585524585847772556157786514009339673616651530269619106162348042904772559024719606855702064792199447996400656006582925642728499731213514708899628591880831009051910970880991064949709247446585585727855519047393381525162179042591094952123288087257673556146588895864807657549871020099063325679239216052627591613871493273124270129972811299321644667304104402131723284762069096766072419150425696190162404199829893699100469545182243987746591657587649544165548475754085335607586293298275639727602304278737245479970202131920950279866120035019301231

In [9]:
f = pow(rand(), r, _p)
root = rth_root(r, h, f)
root

1094742926744099299717467400904595997541503991258375718877709479170031294619331030769224309785176348484107568577402

# Computing final exponentiation witness

We need to first find a proper scalar $\lambda$ such that $c^{\lambda} = f \times \omega_{i}$

In [36]:
Fp12 = GF(p ** 12, 'w', modulus = X ** 12 - 2 * (X ** 6) + 2)
w, z = Fp12(1), Fp12(1)

In [37]:
w, z = Fp12(1), Fp12(1)
s = 2
t = (p^12 - 1) // 3^s
while w == Fp12(1):
    print(w)
    legendre = Fp12(1)
    while legendre == Fp12(1):
        z = Fp12.random_element()
        legendre = z ** (3 * t)
        print(z)
    w = z ** t

1
3098169009852252480582238481732469768803225891824555235709686017996059452964121268396706178497410802686565982296486*w^11 + 801158257592451744910930085908946730132791033349464608357809388015553572406299876783309318257859889270990516623452*w^10 + 2862386960140290892592275214155591187967760793603865004013398372460043170010247395926697630406267117487092135436766*w^9 + 2589158786599828565127973259390697362591138191309483860487141028127315803710125329235176126793806525767232558014286*w^8 + 3616901684495026808040440933871535178019442283321338570167324911990611106689211274300123262162262668495675322791602*w^7 + 782511552588373393373516272005507250001108167890398778173941657742567860113152844170868154251900640260122368370732*w^6 + 2488951069789031182809160652675202626109000580308022283761193398277302431020291186010501751373841844994139679551626*w^5 + 916641657641114260523654235187326942063710512792128016989734156282507055708272787191115805698471450806295222588028*w^4 + 17339318529257125704914

In [42]:
assert(w ** (3^(s - 1) * t) != Fp12(1))
assert(w ** h == Fp12(1))
## just two option, w and w^2, since w^3 must be cubic residue, leading f*w^3 must not be cubic residue
wi = w
if (f * w) ** (3^(s - 1) * t) != Fp12(1):
    assert((f * w^2) ** (3^(s - 1) * t) == Fp12(1))
    wi = w^2
assert(wi ** h == Fp12(1))
f1 = f * wi

AssertionError: 

In [29]:
w ** (3^(s - 1) * t)

1

In [54]:
w ** h

1